# استفاده از RAG با سرویس عامل Azure AI و Semantic Kernel

این قطعه کد نشان می‌دهد که چگونه می‌توان یک عامل Azure AI برای تولید تقویت‌شده با بازیابی (RAG) با استفاده از `Azure AI Agent Service` و `Semantic Kernel` ایجاد و مدیریت کرد. این عامل پرسش‌های کاربران را بر اساس زمینه بازیابی‌شده پردازش کرده و پاسخ‌های دقیقی ارائه می‌دهد.


رفع مشکل نسخه SQLite  
اگر با خطای زیر مواجه شدید:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

این بخش کد را در ابتدای دفترچه یادداشت خود از حالت کامنت خارج کنید:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### وارد کردن بسته‌ها  
کد زیر بسته‌های مورد نیاز را وارد می‌کند:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# تولید مبتنی بر بازیابی با Semantic Kernel و سرویس عامل Azure AI

این نمونه نشان می‌دهد که چگونه می‌توان از **سرویس عامل Azure AI** برای انجام **تولید مبتنی بر بازیابی (RAG)** با ترکیب یک مدل زبانی و زمینه خاص دامنه از یک سند آپلود شده استفاده کرد.

### نحوه عملکرد

1. **آپلود سند**: یک فایل مارک‌داون (document.md) که حاوی اطلاعاتی (سیاست بیمه سفر Contoso) است، به سرویس عامل آپلود می‌شود.

2. **ایجاد ذخیره‌سازی برداری**: سند به یک ذخیره‌سازی برداری شاخص‌گذاری می‌شود تا امکان جستجوی معنایی در محتوای آن فراهم شود.

3. **پیکربندی عامل**: یک عامل با استفاده از مدل `gpt-4o` با دستورالعمل‌های سختگیرانه زیر ایجاد می‌شود:
   - فقط به سوالاتی پاسخ دهد که بر اساس محتوای بازیابی شده از سند هستند.
   - اگر سوال خارج از محدوده باشد، از پاسخ دادن خودداری کند.

4. **ادغام ابزار جستجوی فایل**: ابزار `FileSearchTool` با عامل ثبت می‌شود، که به مدل امکان جستجو و بازیابی قطعات مرتبط از سند شاخص‌گذاری شده در طول استنتاج را می‌دهد.

5. **تعامل کاربر**: کاربران می‌توانند سوالاتی بپرسند. اگر اطلاعات مرتبط در سند یافت شود، عامل پاسخی مبتنی بر سند تولید می‌کند.  
   اگر اطلاعات کافی در سند موجود نباشد، عامل به صراحت پاسخ می‌دهد که سند اطلاعات کافی را در بر ندارد.


### تابع اصلی

تابع اصلی نقطه شروع برنامه است. این تابع معمولاً مسئول مقداردهی اولیه و اجرای منطق اصلی برنامه است.

#### نحوه استفاده
برای استفاده از تابع اصلی، کافی است آن را فراخوانی کنید. در اینجا یک مثال ساده آورده شده است:

```python
def main():
    print("برنامه شروع شد!")
```

#### نکات مهم
[!NOTE] تابع اصلی باید همیشه در انتهای فایل تعریف شود تا خوانایی کد حفظ شود.

#### هشدارها
[!WARNING] اطمینان حاصل کنید که تمام متغیرها و منابع قبل از خروج از تابع اصلی آزاد شده‌اند.

#### نکات مفید
[!TIP] از دستور `if __name__ == "__main__":` برای اجرای مستقیم فایل استفاده کنید.

#### اهمیت
[!IMPORTANT] تابع اصلی باید ساده و قابل فهم باشد تا نگهداری کد آسان‌تر شود.

#### احتیاط
[!CAUTION] از اجرای عملیات سنگین در تابع اصلی خودداری کنید. این کار می‌تواند عملکرد برنامه را تحت تأثیر قرار دهد.


اطمینان حاصل کنید که ابتدا با استفاده از Azure CLI فرمان `az login` را اجرا کنید تا زمینه احراز هویت مناسب هنگام استفاده از `DefaultAzureCredential` فراهم شود. سرویس Azure AI Agent از کلیدهای API استفاده نمی‌کند.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

متن مورد نظر را برای ترجمه ارائه دهید تا بتوانم آن را به فارسی ترجمه کنم.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادقتی‌هایی باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
